# Web scraping

## Co je web scraping?
- Strojové čtení nestrukturovaných dat z webových stránek

## Co není web scraping?
- Stahování dat přes API
- Stahování strukturovaných dat (JSON, CSV,...)
- Crawling - procházení a indexování celé webové stránky pomocí jejích vnitřních hypertextových odkazů


## Příklady web scrapingu

### Analýza cen na českých e-shopech


<div>
    <img src="static/hlidacshopu.png", width="50%"/>
</div>

https://medium.com/@jakubbalada/black-friday-2019-s-hl%C3%ADda%C4%8Dem-shop%C5%AF-9a3ddd352a8c

### Jak hodnotila filmy Mirka Spáčilová


<div>
<img src="static/spacilova.png", width="50%"/>
</div>

https://www.michalblaha.cz/2017/10/filmova-kriticka-mirka-spacilova-v-cislech/

## Etika web scrapingu

- Než začneš s web scrapingem, podívej se, jestli stránka nenabízí strukturovaná data ke stažení nebo neposkytuje API. 
    - **Příklady:** 
    - https://data.gov.cz/datov%C3%A9-sady?poskytovatel=%C4%8Cesk%C3%BD%20statistick%C3%BD%20%C3%BA%C5%99ad
    - https://www.ncdc.noaa.gov/data-access
    - https://www.mapakriminality.cz/data/
    - http://opendata.praha.eu/dataset/meteostanice-chmi-api
- Zjisti si, jaká máš práva k datům, nepublikuj získaná data nelegálně
- Přistupuj ke stránce k rozumné míře, nesnažíš se stránku shodit, ale získat data :-) 

## Z čeho se skládá webová stránka
- **HTML** (HyperText Markup Language): strukturovaný obsah stránky (text a obrázky)
- **CSS** (Cascading Style Sheets): úprava vzhledu stránky
- **JavaScript**: interaktivita obsahu a vzhledu stránky

### HTML 
<div>
    <img src="static/html.png", width="80%">
</div>

- Je tvořen HTML značkami / tagy, např.  ``<img>``
- Většina HTML tagů je párová, např. ``<h2>`` a ``</h2>``
- Tagy mohou mít atributy, které dále specifikují, co a jak bude tag zobrazovat
- Atribut class se obvykle používá k stylování stránky a často podle něj můžeme při webscrapingu odlišit různé části stránky

### CSS

- Popisuje způsob zobrazení html elementů
- Obsahuje 2 části: selektor elementu a blok deklarace:

```
p.error {
  color: red;
}
```


## Pandas: read_html()

Pokud nám stačí stáhnout tabulky z html dokumentu, můžeme použít knihovnu Pandas:

In [1]:
import pandas as pd

In [2]:
tables = pd.read_html('https://cs.wikipedia.org/wiki/Seznam_st%C3%A1t%C5%AF_sv%C4%9Bta_podle_spot%C5%99eby_alkoholu')

In [3]:
len(tables)

2

In [4]:
tables[0].head()

,stát,evidováno,neevidováno,celkem,pivo,víno,destiláty,ostatní
0,Česko,14.97,1.48,16.45,8.51,2.33,3.59,0.39
1,Maďarsko,12.27,4.00,16.27,4.42,4.94,3.02,0.14
2,Rusko,11.03,4.73,15.76,3.65,0.10,6.88,0.34
3,Ukrajina,8.10,7.50,15.60,2.69,0.58,5.21,0.02
4,Estonsko,13.77,1.80,15.57,5.53,1.09,9.19,0.43


In [5]:
tables[1].head()

,Pořadí,Stát,Spotřeba v litrech,Rok
0,1,Francie,12.6,2011
1,2,Rakousko,12.2,2009
2,3,Estonsko,12.0,2011
3,4,Německo,11.7,2009
4,5,Irsko,11.6,2011


## Bonus: co když chceme tabulku z pdf?

Dostat data z pdf dokumentu bývá obtížné. Můžeme zkusit knihovnu [camelot](https://camelot-py.readthedocs.io/en/master/)

In [1]:
import camelot

In [21]:
tables = camelot.read_pdf('static/prd2014.pdf', pages='1-end')

In [22]:
tables

<TableList n=2>

In [23]:
tables[0].df # převedeme tabulku do pandas dataframu

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Start. \nčíslo,Účastník\n(Příjmení Jméno),Pohlaví \n[Z/M],Rok \nnarození,Družstvo,Hmotnost \n[kg],Odhad času \nvítěze\n(h:mm:s),Čas \n[min],Čas \n[sec],Počet \npiv,Relativ\n[H:M:S],Výsledný čas \n[H:M:S],Pořadí \nRELATIV,Pořadí \nABSOLUTNÍ,Vyhodnoc\není \nodhadu
1,1,Jiřičný Martin,M,1969,Hvězdná pěchota,91,0:35:50,44,39,4,0:25:06,0:44:39,3,26,0:00:37
2,2,Cipl František,M,1951,Rychlá včelka,65,0:35:10,53,19,4,0:44:29,0:53:19,89,96,0:01:17
3,3,Míšek Jan,M,1960,,90,0:36:25,56,33,0,0:43:59,0:56:33,83,120,0:00:02
4,4,Němec Miloš,M,1959,Sebranka,73,0:40:50,49,9,4,0:35:37,0:49:09,33,62,0:04:23
5,5,Valko Stanislav,M,1977,To je jedno,84,0:34:02,61,46,2,0:46:28,1:01:46,100,149,0:02:25
6,6,Přerovský Ondřej,M,1972,,79,0:35:00,54,59,0,0:48:43,0:54:59,113,108,0:01:27
7,7,Bareš Jiří,M,1985,,83,0:28:00,53,21,3,0:37:24,0:53:21,46,98,0:08:27
8,8,Rosen Jan,M,1952,,78,0:00:00,56,13,4,0:39:40,0:56:13,62,117,0:36:27
9,9,Hruša Zdeněk,M,1946,,65,0:29:45,49,26,0,0:53:14,0:49:26,135,65,0:06:42


## BeautifulSoup

Knihovna [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) se používá k získávání dat z HTML a XML souborů. Pracuje s různými parsery, které analyzují HTML soubory, a umožnuje vybrat požadované HTML elementy a pracovat s nimi.


In [6]:
from bs4 import BeautifulSoup

In [7]:
# vzorový html
html_doc = '''
<html lang="en-US">
<head>
<meta charset="UTF-8">
<title>PyData Prague | pydata.cz</title>
<link rel="stylesheet" href="/assets/css/style.css?v=3e86527de11985ac075350329bdb8666892c0b1f">
</head>
<body>
<div class="container-lg px-3 my-5 markdown-body"> 
<h1><a href="https://pydata.cz/">pydata.cz</a></h1>
<h1 id="pydata-prague">PyData Prague</h1>
<p id="description">PyData Prague is a community of data scientists, engineers, analysts, and various other developers 
in the area of scientific computing and data analysis. The term <a class="pydata" href="https://pydata.org/">PyData</a> 
refers to an educational program of <a href="https://numfocus.org/">NumFOCUS</a>, an american non-profit 
helping open source software in terms of governance, financial support, and operations.</p>
<p>The PyData network hosts meetups in hundreds of cities around the world and several conferences each 
year. The Prague chapter started in 2018 with the aim of spreading the word of open source scientific 
computing in the Czech Republic. And while the chapter is based in Prague, we operate and 
collaborate countrywide.</p>
<h2 id="code-of-conduct">Code of Conduct</h2>
<p>We adhere to PyData’s code of conduct, here’s its short version:</p>
<blockquote>
<p>Be kind to others. Do not insult or put down others. Behave professionally. 
Remember that harassment and sexist, racist, or exclusionary jokes and language are not 
appropriate for PyData.</p>
<p>All communication should be appropriate for a professional audience including 
people of many different backgrounds. Sexual language and imagery is not appropriate.</p>
<p>PyData is dedicated to providing a harassment-free event experience for everyone, regardless of 
gender, sexual orientation, gender identity, and expression, disability, physical appearance, 
body size, race, or religion. We do not tolerate harassment of participants in any form.</p>
<p>Thank you for helping make this a welcoming, friendly community for all.</p>
</blockquote>
<p>You can find more information at 
<a class="pydata" href="https://pydata.org/code-of-conduct/">pydata.org/code-of-conduct/</a></p 
</div>  
</body>
</html>
'''

In [8]:
soup = BeautifulSoup(html_doc, 'html.parser')
# Vytiskneme hezky zformatované HTML
print(soup.prettify())

<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <title>
   PyData Prague | pydata.cz
  </title>
  <link href="/assets/css/style.css?v=3e86527de11985ac075350329bdb8666892c0b1f" rel="stylesheet"/>
 </head>
 <body>
  <div class="container-lg px-3 my-5 markdown-body">
   <h1>
    <a href="https://pydata.cz/">
     pydata.cz
    </a>
   </h1>
   <h1 id="pydata-prague">
    PyData Prague
   </h1>
   <p id="description">
    PyData Prague is a community of data scientists, engineers, analysts, and various other developers 
in the area of scientific computing and data analysis. The term
    <a class="pydata" href="https://pydata.org/">
     PyData
    </a>
    refers to an educational program of
    <a href="https://numfocus.org/">
     NumFOCUS
    </a>
    , an american non-profit 
helping open source software in terms of governance, financial support, and operations.
   </p>
   <p>
    The PyData network hosts meetups in hundreds of cities around the world and several conferences ea

In [9]:
soup.title

<title>PyData Prague | pydata.cz</title>

In [10]:
# ukáže první element typu h1 
soup.h1

<h1><a href="https://pydata.cz/">pydata.cz</a></h1>

In [11]:
soup.h1.name

'h1'

In [12]:
soup.h1.parent.name

'div'

In [13]:
soup.h1.string

'pydata.cz'

In [14]:
soup.h2

<h2 id="code-of-conduct">Code of Conduct</h2>

In [15]:
soup.h2['id']

'code-of-conduct'

In [16]:
# najde všechny odkazy
links = soup.find_all('a')
links

[<a href="https://pydata.cz/">pydata.cz</a>,
 <a class="pydata" href="https://pydata.org/">PyData</a>,
 <a href="https://numfocus.org/">NumFOCUS</a>,
 <a class="pydata" href="https://pydata.org/code-of-conduct/">pydata.org/code-of-conduct/</a>]

In [17]:
links[0]['href']

'https://pydata.cz/'

In [18]:
links[0].text

'pydata.cz'

In [19]:
soup.find(id="pydata-prague")

<h1 id="pydata-prague">PyData Prague</h1>

Pokud hledáme podle třídy, memůžeme použít ``soup.find(class="pydata")``, protože ``class`` je v Pythonu klíčové slovo. Musíme použít ``class_``

In [20]:
soup.find(class_="pydata")

<a class="pydata" href="https://pydata.org/">PyData</a>

``class`` můžeme taky napsat jako klíč v parametru ``attrs``:

In [21]:
soup.find(attrs={'class':'pydata'})

<a class="pydata" href="https://pydata.org/">PyData</a>

### Cvičení


Vyberte odstavec s id ``description``.

Vyberte všechny odkazy s třídou ``pydata``

## Requests

Knihovna [requests](https://requests.readthedocs.io/en/master/) je určená pro HTTP dotazy. V našem případě ji budeme používat pro získání textu webové stránky.


In [22]:
import requests

In [23]:
r = requests.get('https://pydata.cz/')

In [24]:
r.status_code

200

## Příklad z neživota

## Nástroje v prohlížeči ?

## Jak na JavaScript